In [1]:
import pandas as pd

In [2]:
from pybaseball import statcast_batter, statcast_pitcher
from pybaseball import statcast_pitcher_percentile_ranks
from pybaseball import pitching_stats
from pybaseball import playerid_lookup
import numpy as np


In [3]:
data = pitching_stats(2023, qual=10)
data.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
187,10310,2023,Zack Wheeler,PHI,33,13,6,5.9,3.61,32,...,NaN,NaN,NaN,NaN,109,106,111,NaN,NaN,NaN
238,27498,2023,Spencer Strider,ATL,24,20,5,5.5,3.86,32,...,NaN,NaN,NaN,NaN,125,104,112,NaN,NaN,NaN
119,14107,2023,Kevin Gausman,TOR,32,12,9,5.3,3.16,31,...,NaN,105.0,108.0,109.0,105,104,105,NaN,NaN,NaN
67,12768,2023,Sonny Gray,MIN,33,8,8,5.3,2.79,32,...,102.0,NaN,NaN,NaN,103,101,104,NaN,NaN,NaN
169,19291,2023,Zac Gallen,ARI,27,17,9,5.2,3.47,34,...,95.0,NaN,NaN,NaN,106,104,107,NaN,NaN,NaN


In [4]:
data.shape

(659, 393)

In [5]:
# Iterate through the names and print them
selected_pitchers = []
for name in data['Name']:
    selected_pitchers.append(name)

last_selected_pitchers = [name.split()[-1] for name in selected_pitchers]
first_selected_pitchers = [name.split()[0] for name in selected_pitchers]

list = zip(first_selected_pitchers, last_selected_pitchers)
result = set(list)
result

{('A.J.', 'Minter'),
 ('A.J.', 'Puk'),
 ('AJ', 'Smith-Shawver'),
 ('Aaron', 'Bummer'),
 ('Aaron', 'Civale'),
 ('Aaron', 'Loup'),
 ('Aaron', 'Nola'),
 ('Abner', 'Uribe'),
 ('Adam', 'Cimber'),
 ('Adam', 'Oller'),
 ('Adam', 'Ottavino'),
 ('Adam', 'Wainwright'),
 ('Adbert', 'Alzolay'),
 ('Adrian', 'Houser'),
 ('Adrian', 'Martinez'),
 ('Albert', 'Abreu'),
 ('Alec', 'Marsh'),
 ('Alek', 'Manoah'),
 ('Alex', 'Cobb'),
 ('Alex', 'Faedo'),
 ('Alex', 'Lange'),
 ('Alex', 'Vesia'),
 ('Alex', 'Wood'),
 ('Alex', 'Young'),
 ('Alexis', 'Diaz'),
 ('Allan', 'Winans'),
 ('Amir', 'Garrett'),
 ('Amos', 'Willingham'),
 ('Andre', 'Jackson'),
 ('Andre', 'Pallante'),
 ('Andres', 'Machado'),
 ('Andres', 'Munoz'),
 ('Andrew', 'Abbott'),
 ('Andrew', 'Bellatti'),
 ('Andrew', 'Chafin'),
 ('Andrew', 'Heaney'),
 ('Andrew', 'Kittredge'),
 ('Andrew', 'Nardi'),
 ('Andrew', 'Saalfrank'),
 ('Andrew', 'Suarez'),
 ('Andrew', 'Vasquez'),
 ('Andrew', 'Wantz'),
 ('Angel', 'Felipe'),
 ('Angel', 'Perdomo'),
 ('Angel', 'Zerpa'),
 (

In [6]:
len(result)

657

In [7]:
player_ids = []

# Iterate through the list of player name tuples
for first_name, last_name in result:
    try:
        # Use playerid_lookup function with first and last names
        player_id_df = playerid_lookup(last_name, first_name)

        # Check if the DataFrame is not empty and add to the list
        if player_id_df is not None and not player_id_df.empty:
            player_ids.append(player_id_df)
    except Exception as e:
        print(f"Error fetching player ID for {first_name} {last_name}: {e}")

# Optional: Combine all the individual DataFrames into one
combined_player_ids = pd.concat(player_ids, ignore_index=True)

Gathering player lookup table. This may take a moment.


In [8]:
combined_player_ids

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,cimber,adam,643256,cimba001,cimbead01,15288,2018.0,2023.0
1,boyle,joe,671212,boylj001,boylejo01,29608,2023.0,2023.0
2,kuhnel,joel,669270,kuhnj001,kuhnejo01,19995,2019.0,2023.0
3,houser,adrian,605288,housa001,housead01,12718,2015.0,2023.0
4,garcia,rico,670329,garcr006,garciri01,20023,2019.0,2023.0
...,...,...,...,...,...,...,...,...
583,peralta,wandy,593974,peraw002,peralwa01,14295,2016.0,2023.0
584,kluber,corey,446372,klubc001,klubeco01,2429,2011.0,2023.0
585,patton,spencer,607359,patts002,pattosp01,12361,2014.0,2023.0
586,jacques,joe,682175,jacqj001,jacqujo01,25178,2023.0,2023.0


In [9]:
combined_player_ids = combined_player_ids['key_mlbam']

In [10]:
combined_player_ids

0      643256
1      671212
2      669270
3      605288
4      670329
        ...  
583    593974
584    446372
585    607359
586    682175
587    674072
Name: key_mlbam, Length: 588, dtype: int64

In [11]:
combined_df = pd.DataFrame()

# Iterate through each pitcher's name and fetch their data
for id in combined_player_ids:
    try:
        # Fetch data for the pitcher
        pitcher_df = statcast_pitcher('2023-04-01', '2023-09-30', id)

        # Check if the DataFrame is not empty
        if pitcher_df is not None and not pitcher_df.empty:
            # Append the data to the combined DataFrame
            combined_df = pd.concat([combined_df, pitcher_df])
    except Exception as e:
        print(f"Error fetching data for {name}: {e}")

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

In [12]:
combined_df.to_csv('../data/pitcher_data.zip', index=False)

In [13]:
combined_df = pd.read_csv('../data/pitcher_data.zip')

In [14]:
combined_df.shape

(610225, 92)

In [15]:
combined_df.isna().sum()

pitch_type                165
game_date                   0
release_speed             176
release_pos_x             168
release_pos_z             168
                         ... 
if_fielding_alignment    2204
of_fielding_alignment    2204
spin_axis                3679
delta_home_win_exp          0
delta_run_exp              48
Length: 92, dtype: int64

In [16]:
combined_df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [17]:
combined_df[['pfx_x', 'pfx_z']]

,pfx_x,pfx_z
0,-0.76,-0.82
1,-0.94,-0.58
2,0.64,1.23
3,-0.67,0.09
4,-0.51,0.23
...,...,...
610220,0.40,1.37
610221,0.60,1.23
610222,1.40,0.39
610223,0.55,1.48


In [18]:
cols3 = ['effective_speed', 'release_spin_rate', 'spin_axis', ]
combined_df = combined_df[cols3]

In [19]:
from pybaseball import statcast_pitcher_spin

In [20]:
spintest_df = get_statcast_pitcher_spin('2023-04-01', '2023-09-30', player_id = 489119)
spintest_df

NameError: name 'get_statcast_pitcher_spin' is not defined